In [42]:
import argparse
import pandas as pd
import numpy as np
import h5torch
import py2bit
import random


def get_chrom_number(chr):
    try:
        num = int(chr[3:])
    except:
        num = 100
    return num

bed_path = "/home/data/shared/natant/Data/remap_merged_peaks.bed"
genome_path = "/home/natant/Thesis-plmbind/Data/Genomes/hg38.2bit"
chrom = [1, 2]
alts = False
NOTchrom = None
all_TFs = "/home/natant/Thesis-plmbind/Thesis/utils/TF_split/TF_data_split/ALL_TFs.txt"
data = pd.read_csv(bed_path, sep = "\t", header = None)
data.head()


TF_list = []
with open(all_TFs) as f:
    for TF in f:
        TF_list.append(TF.strip())
              
prot_db = dict(zip(TF_list, range(len(TF_list))))

tb = py2bit.open(genome_path)

# define mapping for nucleotides:
mapping = {"A": 0, "T": 1, "C": 2, "G": 3, "N": 4}

In [43]:
Unique_chrom_names = list(tb.chroms())
chrom_order = sorted(list(Unique_chrom_names), key=lambda x: get_chrom_number(x))
print("Used chromosomes: ")
if chrom != None:
    chroms = [chrom_order[i-1] for i in chrom]
    if alts == "True":
        chroms = [i for i in chrom_order if i.split("_")[0] in chroms]
    print(chroms)
elif NOTchrom != None:
    NOTchroms = [chrom_order[i-1] for i in NOTchrom]
    special_chroms = [i for i in chrom_order if i.split("_")[0] in NOTchroms]
    NOTchroms = NOTchroms + special_chroms
    chroms_w_alts = [i for i in chrom_order if i not in NOTchroms]
    chroms = [i for i in chroms_w_alts if len(i.split("_"))==1]
    if alts == "True":
        chroms = chroms_w_alts[:]
        
    print(chroms)
else:
    raise SystemExit(1)

Used chromosomes: 
['chr1', 'chr2']


In [44]:
sequence_per_chrom = []
rows_per_chrom = []
cols_per_chrom = []
DNA_frames = []
rows = []
cols = []
for chrom_name in chroms:
    chrom_rows = []
    print(chrom_name)
    print('gettin chrom ..')
    # get peaks for chrom
    data_subset = data[data[0] == chrom_name]
  
    desired_length = 2048
    
    for ix, i in enumerate(range(len(data_subset))):
        start, stop, TFs = data_subset.iloc[i][[1, 2, 3]]
        delta = stop-start
        if delta <= desired_length:
            if delta < desired_length:
                # NOG CHECKEN DAT HET JUIST IS
                prefix = random.randint(0,desired_length-delta)
                suffix = desired_length-delta-prefix
                start = start - prefix
                stop = stop + suffix
            
            # HIER
            chrom_rows.append((chrom_name, start, stop))
            cols.append([prot_db[i] for i in TFs.split(",")])
            
    chrom = np.array([mapping[l] for l in tb.sequence(chrom_name)], dtype=np.int8)
    DNA_frames.extend([chrom[i[1]:i[2]] for i in chrom_rows])
    rows.extend(chrom_rows)

chr1
gettin chrom ..
chr2
gettin chrom ..


In [45]:
y = np.zeros((len(rows), len(prot_db)), dtype=np.bool_)
y.shape
for i,TFs in enumerate(cols):
    y[i,TFs] = True
    
from scipy.sparse import csr_matrix
mat = csr_matrix(y) # Die matrix dan sparse maken
mat.indices = mat.indices.astype(np.int16) 

In [46]:
mat

<476292x1210 sparse matrix of type '<class 'numpy.bool_'>'
	with 5539120 stored elements in Compressed Sparse Row format>

In [47]:
len(DNA_frames)

476292

In [48]:
np.array(DNA_frames).shape

(476292, 2048)

In [49]:
f = h5torch.File("/home/natant/Thesis-plmbind/Thesis/utils/data_processing/test.h5", "w")
f.register(mat, axis="central", mode = "csr", dtype_save = "bool", dtype_load="int64")
f.register(np.array(DNA_frames), axis = 0, name = "DNA", mode = "N-D", dtype_save = "int8", dtype_load="int64")
ix_to_prot = {v : k for k, v in prot_db.items()}
prot_mapping = np.array([ix_to_prot[i] for i in range(len(prot_db))]).astype(bytes)
f.register(prot_mapping, axis = 1, name = "prots", mode = "N-D", dtype_save = "bytes", dtype_load = "str")
ix_to_nuc = {v : k for k, v in mapping.items()}
nucleotide_mapping = np.array([ix_to_nuc[i] for i in range(len(ix_to_nuc))]).astype(bytes)
f.register(nucleotide_mapping, axis = "unstructured", name = "nucleotide_mapping", mode = "N-D", dtype_save = "bytes", dtype_load = "str")

f.visititems(print)
f.close()


0 <HDF5 group "/0" (1 members)>
0/DNA <HDF5 dataset "DNA": shape (476292, 2048), type "|i1">
1 <HDF5 group "/1" (1 members)>
1/prots <HDF5 dataset "prots": shape (1210,), type "|S13">
central <HDF5 group "/central" (3 members)>
central/data <HDF5 dataset "data": shape (5539120,), type "|b1">
central/indices <HDF5 dataset "indices": shape (5539120,), type "<i4">
central/indptr <HDF5 dataset "indptr": shape (476293,), type "<i4">
unstructured <HDF5 group "/unstructured" (1 members)>
unstructured/nucleotide_mapping <HDF5 dataset "nucleotide_mapping": shape (5,), type "|S1">
